In [40]:
import pandas as pd
import numpy as np

In [41]:
df=pd.read_csv("0_tube_stock.csv")

In [42]:
iteration=0
dim=7

In [43]:
re1=["A1","A2","A3","A4","A5","A6","B1","B2","B3"]
re2=["C1","C2","C3","C4","C5","C6","D1","D2","D3"]
racknumber=int(np.ceil(dim/2))
rackcode={"Glucose":("1","1"),"NH4Cl":("1","2"),"MgSO4":("2","1"),"KH2PO4":("2","2"),
        "Na2HPO4":("3","1"),"CaCl2":("3","2"),"NaCl": ("4","1")}
bigcode={"Glucose":"A1","NH4Cl":"A2","MgSO4":"A3","KH2PO4":"A4",
        "Na2HPO4":"A5","CaCl2":"A6","NaCl": "B1"}

In [47]:
with open("../opentrons_scripts/"+str(iteration)+"_stock_opentrons.py","w") as f:
    f.write("from opentrons import protocol_api\nmetadata = {'apiLevel': '2.13'}\ndef run(protocol: protocol_api.ProtocolContext):\n")
    f.write("\ttiprack_2 = protocol.load_labware('opentrons_96_tiprack_300ul', 11)\n\ttiprack_1 = protocol.load_labware('opentrons_96_tiprack_1000ul',10)\n")
    for rn in range(1, racknumber+1):
        f.write("\treservoir_"+str(rn)+"= protocol.load_labware('opentrons_24_tuberack_eppendorf_2ml_safelock_snapcap',"+str(rn)+")\n")
    f.write("\tbig= protocol.load_labware('corning_12_wellplate_6.9ml_flat',9)\n\tbig2= protocol.load_labware('corning_12_wellplate_6.9ml_flat',7)\n\tp300 = protocol.load_instrument('p300_single', 'left', tip_racks=[tiprack_2])\n\tp1000 = protocol.load_instrument('p1000_single', 'right', tip_racks=[tiprack_1])\n")
    for i in pd.unique(df.columns[1:]):
        df2=df[["sample",i]].copy().reset_index(drop=True)
        f.write("\t#"+i+"\n")
        if (df2[i] <= 300).all():
            f.write("\tp300.pick_up_tip()\n")
        elif (df2[i] > 300).all():
            f.write("\tp1000.pick_up_tip()\n") 
        else:
            f.write("\tp300.pick_up_tip()\n")
            f.write("\tp1000.pick_up_tip()\n")
        cati=i.split("_")
        rackdata=rackcode[cati[1]]
        if rackdata[1]=="1":
            linea=re1
        else:
            linea=re2
        if cati[0]=="Vol":
            for iq,j in df2.iterrows():
                if ((j[i]<=300) & (j[i]!=0)):
                    f.write("\tp300.aspirate("+str(j[i])+",big['"+bigcode[cati[1]]+"'],rate=0.7)\n")
                    f.write("\tp300.dispense("+str(j[i])+",reservoir_"+rackdata[0]+"['"+linea[iq]+"'],rate=0.7)\n")
                if ((j[i]>300) & (j[i]<=1000)):
                    f.write("\tp1000.aspirate("+str(j[i])+",big['"+bigcode[cati[1]]+"'],rate=0.7)\n")
                    f.write("\tp1000.dispense("+str(j[i])+",reservoir_"+rackdata[0]+"['"+linea[iq]+"'],rate=0.7)\n")
                if ((j[i]>1000)):
                    newfor="{:.2f}".format(j[i]/2)
                    f.write("\tp1000.aspirate("+newfor+",big['"+bigcode[cati[1]]+"'],rate=0.7)\n")
                    f.write("\tp1000.dispense("+newfor+",reservoir_"+rackdata[0]+"['"+linea[iq]+"'],rate=0.7)\n")
                    f.write("\tp1000.aspirate("+newfor+",big['"+bigcode[cati[1]]+"'],rate=0.7)\n")
                    f.write("\tp1000.dispense("+newfor+",reservoir_"+rackdata[0]+"['"+linea[iq]+"'],rate=0.7)\n")
        else:
            for iq,j in df2.iterrows():
                if ((j[i]<=300) & (j[i]!=0)):
                    f.write("\tp300.aspirate("+str(j[i])+",big2['A2'],rate=0.7)\n")
                    f.write("\tp300.dispense("+str(j[i])+",reservoir_"+rackdata[0]+"['"+linea[iq]+"'],rate=0.7)\n")
                if ((j[i]>300) & (j[i]<=1000)):
                    f.write("\tp1000.aspirate("+str(j[i])+",big2['A2'],rate=0.7)\n")
                    f.write("\tp1000.dispense("+str(j[i])+",reservoir_"+rackdata[0]+"['"+linea[iq]+"'],rate=0.7)\n")
                if ((j[i]>1000)):
                    newfor="{:.2f}".format(j[i]/2)
                    f.write("\tp1000.aspirate("+newfor+",big['"+bigcode[cati[1]]+"'],rate=0.7)\n")
                    f.write("\tp1000.dispense("+newfor+",reservoir_"+rackdata[0]+"['"+linea[iq]+"'],rate=0.7)\n")
                    f.write("\tp1000.aspirate("+newfor+",big['"+bigcode[cati[1]]+"'],rate=0.7)\n")
                    f.write("\tp1000.dispense("+newfor+",reservoir_"+rackdata[0]+"['"+linea[iq]+"'],rate=0.7)\n")
        if (df2[i] <= 300).all():
            f.write("\tp300.drop_tip()\n")
        elif (df2[i] >= 1000).all():
            f.write("\tp1000.drop_tip()\n") 
        else:
            f.write("\tp300.drop_tip()\n")
            f.write("\tp1000.drop_tip()\n")

In [45]:
df

,sample,Vol_Glucose,Water_Glucose,Vol_NH4Cl,Water_NH4Cl,Vol_MgSO4,Water_MgSO4,Vol_KH2PO4,Water_KH2PO4,Vol_Na2HPO4,Water_Na2HPO4,Vol_CaCl2,Water_CaCl2,Vol_NaCl,Water_NaCl
0,0_1,667.4,1332.6,271.0,1729.0,2000.0,0.0,880.0,1120.0,1692.0,308.0,1000.0,1000.0,342.0,1658.0
1,0_2,1147.5,852.5,1054.1,945.9,2000.0,0.0,880.0,1120.0,1692.0,308.0,1000.0,1000.0,342.0,1658.0
2,0_3,1515.9,484.1,601.7,1398.3,2000.0,0.0,880.0,1120.0,1692.0,308.0,1000.0,1000.0,342.0,1658.0
3,0_4,43.2,1956.8,1822.5,177.5,2000.0,0.0,880.0,1120.0,1692.0,308.0,1000.0,1000.0,342.0,1658.0
4,0_5,267.6,1732.4,779.9,1220.1,2000.0,0.0,880.0,1120.0,1692.0,308.0,1000.0,1000.0,342.0,1658.0
5,0_6,1795.3,204.7,1559.1,440.9,2000.0,0.0,880.0,1120.0,1692.0,308.0,1000.0,1000.0,342.0,1658.0
6,0_7,1415.2,584.8,97.4,1902.6,2000.0,0.0,880.0,1120.0,1692.0,308.0,1000.0,1000.0,342.0,1658.0
7,0_CTRL,200.0,1800.0,155.8,1844.2,2000.0,0.0,880.0,1120.0,1692.0,308.0,1000.0,1000.0,342.0,1658.0
8,0_REF,200.0,1800.0,155.8,1844.2,2000.0,0.0,880.0,1120.0,1692.0,308.0,1000.0,1000.0,342.0,1658.0
